In [1]:
import torch

In [2]:
from vae import test_nll_estimation, parse_arguments

In [3]:
from pathlib import Path

In [4]:
args = dict(
    data_path='../Data/Dataset',
    device='cuda' if torch.cuda.is_available() else 'cpu',
    embedding_size=300,
    hidden_size=256,
    latent_size=16,
    num_layers=1,
    word_dropout=1.0,
    freebits=None,
    model_save_path='models',
    batch_size_valid=64,
    num_samples=10,
)

In [11]:
def get_approx_nlls(saved_model_files):
    nlls = []
    for saved_model_file in saved_model_files:
        nll = test_nll_estimation(saved_model_file=saved_model_file, **args)
        nlls.append(nll)
    return nlls

In [6]:
def get_model_filenames(experiment_name='vanilla'):
    saved_model_files = [next(Path(f"results_final/results{i}/{experiment_name}/models/").iterdir()) for i in range(4)]
    saved_model_files = [str(f) for f in saved_model_files]
    return saved_model_files

In [7]:
num_samples = 10

# Vanilla NLL

In [8]:
saved_model_files = get_model_filenames('vanilla')
saved_model_files

['results_final/results0/vanilla/models/sentence_vae_3500.pt',
 'results_final/results1/vanilla/models/sentence_vae_3500.pt',
 'results_final/results2/vanilla/models/sentence_vae_3500.pt',
 'results_final/results3/vanilla/models/sentence_vae_3500.pt']

In [12]:
vanilla_nlls = get_approx_nlls(saved_model_files)

  0%|          | 0/10 [00:00<?, ?it/s]
                                      
  0%|          | 0/38 [00:00<?, ?it/s]
                                      
 20%|██        | 2/10 [00:00<00:00, 18.54it/s]
                                              
  0%|          | 0/38 [00:00<?, ?it/s].54it/s]
                                              
  0%|          | 0/38 [00:00<?, ?it/s].54it/s]
                                              
 50%|█████     | 5/10 [00:00<00:00, 20.02it/s]

nll mean: 117.48198699951172 	 kl mean: 0.04554252326488495 	 loss mean: 117.52752685546875
nll mean: 117.47480773925781 	 kl mean: 0.04554252326488495 	 loss mean: 117.52035522460938
nll mean: 117.46318817138672 	 kl mean: 0.04554252326488495 	 loss mean: 117.50872802734375
nll mean: 117.3797607421875 	 kl mean: 0.04554252326488495 	 loss mean: 117.42530059814453
nll mean: 117.45420837402344 	 kl mean: 0.04554252326488495 	 loss mean: 117.499755859375



                                              
  0%|          | 0/38 [00:00<?, ?it/s].02it/s]
                                              
  0%|          | 0/38 [00:00<?, ?it/s].02it/s]
                                              
 80%|████████  | 8/10 [00:00<00:00, 21.22it/s]
                                              
  0%|          | 0/38 [00:00<?, ?it/s].22it/s]
                                              
  0%|          | 0/10 [00:00<?, ?it/s]

nll mean: 117.49592590332031 	 kl mean: 0.04554252326488495 	 loss mean: 117.54146575927734
nll mean: 117.4547119140625 	 kl mean: 0.04554252326488495 	 loss mean: 117.50025939941406
nll mean: 117.44595336914062 	 kl mean: 0.04554252326488495 	 loss mean: 117.49149322509766
nll mean: 117.48948669433594 	 kl mean: 0.04554252326488495 	 loss mean: 117.53501892089844
nll mean: 117.43524169921875 	 kl mean: 0.04554252326488495 	 loss mean: 117.48077392578125


                                              
  3%|▎         | 1/38 [00:00<00:16,  2.31it/s]
                                              
  0%|          | 0/10 [00:00<?, ?it/s]
                                              
  3%|▎         | 1/38 [00:00<00:16,  2.31it/s]
                                              
 40%|████      | 4/10 [00:00<00:00, 14.96it/s]

...

nll mean: 117.71975708007812 	 kl mean: 0.07238086313009262 	 loss mean: 117.79212951660156
nll mean: 117.64869689941406 	 kl mean: 0.07238086313009262 	 loss mean: 117.7210693359375
nll mean: 117.75633239746094 	 kl mean: 0.07238086313009262 	 loss mean: 117.8287124633789
nll mean: 117.61582946777344 	 kl mean: 0.07238086313009262 	 loss mean: 117.68820190429688
nll mean: 117.740966796875 	 kl mean: 0.07238086313009262 	 loss mean: 117.8133544921875


                                               
 97%|█████████▋| 37/38 [00:23<00:00,  1.53it/s]
                                               
 97%|█████████▋| 37/38 [00:23<00:00,  1.53it/s]
                                               
 97%|█████████▋| 37/38 [00:23<00:00,  1.53it/s]
                                               
 97%|█████████▋| 37/38 [00:23<00:00,  1.53it/s]
                                               
  0%|          | 0/10 [00:00<?, ?it/s]
                                               [A
 97%|█████████▋| 37/38 [00:23<00:00,  1.53it/s]
                                               [A
 40%|████      | 4/10 [00:00<00:00, 31.54it/s]

nll mean: 117.62576293945312 	 kl mean: 0.07238086313009262 	 loss mean: 117.69813537597656
nll mean: 91.02618408203125 	 kl mean: 0.06083798408508301 	 loss mean: 91.08702087402344
nll mean: 91.06071472167969 	 kl mean: 0.06083798408508301 	 loss mean: 91.12154388427734
nll mean: 90.98592376708984 	 kl mean: 0.06083798408508301 	 loss mean: 91.04676818847656
nll mean: 90.9239730834961 	 kl mean: 0.06083798408508301 	 loss mean: 90.98480224609375
nll mean: 90.93962097167969 	 kl mean: 0.06083798408508301 	 loss mean: 91.00045776367188
nll mean: 90.9952621459961 	 kl mean: 0.06083798408508301 	 loss mean: 91.05609130859375


                                               
 97%|█████████▋| 37/38 [00:23<00:00,  1.53it/s]
                                               [A
 40%|████      | 4/10 [00:00<00:00, 31.54it/s]
                                               [A
 97%|█████████▋| 37/38 [00:23<00:00,  1.53it/s]
                                               [A
100%|██████████| 38/38 [00:23<00:00,  1.61it/s]

nll mean: 91.0322265625 	 kl mean: 0.06083798408508301 	 loss mean: 91.09307098388672
nll mean: 91.01310729980469 	 kl mean: 0.06083798408508301 	 loss mean: 91.07393646240234
nll mean: 90.98198699951172 	 kl mean: 0.06083798408508301 	 loss mean: 91.04283142089844
nll mean: 91.12106323242188 	 kl mean: 0.06083798408508301 	 loss mean: 91.1819076538086
Approximate NLL:
tensor(126.7324, device='cuda:0')
Testing took 0:00:24.485483


In [13]:
print(vanilla_nlls)

[tensor(126.9911, device='cuda:0'), tensor(127.1313, device='cuda:0'), tensor(127.0736, device='cuda:0'), tensor(126.7324, device='cuda:0')]


# Word dropout

In [14]:
saved_model_files = get_model_filenames('word_dropout_066')
saved_model_files

['results_final/results0/word_dropout_066/models/sentence_vae_6000.pt',
 'results_final/results1/word_dropout_066/models/sentence_vae_4500.pt',
 'results_final/results2/word_dropout_066/models/sentence_vae_6000.pt',
 'results_final/results3/word_dropout_066/models/sentence_vae_6000.pt']

In [15]:
wd_nlls = get_approx_nlls(saved_model_files)

  0%|          | 0/38 [00:00<?, ?it/s]
                                      
  0%|          | 0/38 [00:00<?, ?it/s]
                                      
  0%|          | 0/38 [00:00<?, ?it/s]
                                      
  0%|          | 0/10 [00:00<?, ?it/s]
                                      .66it/s]
  0%|          | 0/38 [00:00<?, ?it/s]        
                                      .66it/s]
  0%|          | 0/38 [00:00<?, ?it/s]        
                                      .66it/s]
 60%|██████    | 6/10 [00:00<00:00, 26.29it/s]

nll mean: 114.30663299560547 	 kl mean: 1.0882575511932373 	 loss mean: 115.39488220214844
nll mean: 114.34886169433594 	 kl mean: 1.0882575511932373 	 loss mean: 115.4371109008789
nll mean: 114.50798797607422 	 kl mean: 1.0882575511932373 	 loss mean: 115.59624481201172
nll mean: 114.55747985839844 	 kl mean: 1.0882575511932373 	 loss mean: 115.64573669433594
nll mean: 114.520263671875 	 kl mean: 1.0882575511932373 	 loss mean: 115.60852813720703
nll mean: 114.43206787109375 	 kl mean: 1.0882575511932373 	 loss mean: 115.52032470703125


                                      
  0%|          | 0/38 [00:00<?, ?it/s]        
                                      .29it/s]
  0%|          | 0/38 [00:00<?, ?it/s]        
                                      .29it/s]
 60%|██████    | 6/10 [00:00<00:00, 26.29it/s]
                                      .98it/s]
  0%|          | 0/10 [00:00<?, ?it/s]
                                              
  3%|▎         | 1/38 [00:00<00:14,  2.56it/s]

nll mean: 114.56140899658203 	 kl mean: 1.0882575511932373 	 loss mean: 115.64967346191406
nll mean: 114.56852722167969 	 kl mean: 1.0882575511932373 	 loss mean: 115.65678405761719
nll mean: 114.53750610351562 	 kl mean: 1.0882575511932373 	 loss mean: 115.62576293945312
nll mean: 114.25630187988281 	 kl mean: 1.0882575511932373 	 loss mean: 115.34456634521484
nll mean: 131.324462890625 	 kl mean: 1.304136037826538 	 loss mean: 132.62860107421875



                                              
 20%|██        | 2/10 [00:00<00:00, 13.82it/s]
                                              
  3%|▎         | 1/38 [00:00<00:14,  2.56it/s]
                                              
 40%|████      | 4/10 [00:00<00:00, 13.93it/s]

...

nll mean: 117.52592468261719 	 kl mean: 1.0752837657928467 	 loss mean: 118.60120391845703
nll mean: 117.31697082519531 	 kl mean: 1.0752837657928467 	 loss mean: 118.39225006103516
nll mean: 117.28692626953125 	 kl mean: 1.0752837657928467 	 loss mean: 118.36221313476562
nll mean: 117.3926773071289 	 kl mean: 1.0752837657928467 	 loss mean: 118.46795654296875
nll mean: 117.29874420166016 	 kl mean: 1.0752837657928467 	 loss mean: 118.37403106689453


                                               
 95%|█████████▍| 36/38 [00:23<00:01,  1.35it/s]
                                               [A
 60%|██████    | 6/10 [00:00<00:00, 22.81it/s]
                                               [A
 97%|█████████▋| 37/38 [00:23<00:00,  1.53it/s]
                                               
 97%|█████████▋| 37/38 [00:23<00:00,  1.53it/s]
                                               
 97%|█████████▋| 37/38 [00:23<00:00,  1.53it/s]
                                               
  0%|          | 0/10 [00:00<?, ?it/s]

nll mean: 117.58059692382812 	 kl mean: 1.0752837657928467 	 loss mean: 118.6558837890625
nll mean: 117.4215087890625 	 kl mean: 1.0752837657928467 	 loss mean: 118.49678802490234
nll mean: 117.51509094238281 	 kl mean: 1.0752837657928467 	 loss mean: 118.59037017822266
nll mean: 89.4488754272461 	 kl mean: 1.7217345237731934 	 loss mean: 91.17061614990234
nll mean: 89.4103012084961 	 kl mean: 1.7217345237731934 	 loss mean: 91.13203430175781
nll mean: 89.62705993652344 	 kl mean: 1.7217345237731934 	 loss mean: 91.34880065917969


                                               
  0%|          | 0/10 [00:00<?, ?it/s]
                                               [A
 97%|█████████▋| 37/38 [00:23<00:00,  1.53it/s]
                                               [A
 97%|█████████▋| 37/38 [00:23<00:00,  1.53it/s]
                                               [A
 97%|█████████▋| 37/38 [00:23<00:00,  1.53it/s]
                                               [A
 40%|████      | 4/10 [00:00<00:00, 31.11it/s]
                                               [A
 97%|█████████▋| 37/38 [00:23<00:00,  1.53it/s]
                                               [A
100%|██████████| 38/38 [00:23<00:00,  1.60it/s]

nll mean: 89.356201171875 	 kl mean: 1.7217345237731934 	 loss mean: 91.07794189453125
nll mean: 89.33430480957031 	 kl mean: 1.7217345237731934 	 loss mean: 91.0560302734375
nll mean: 89.34600067138672 	 kl mean: 1.7217345237731934 	 loss mean: 91.06773376464844
nll mean: 89.38018798828125 	 kl mean: 1.7217345237731934 	 loss mean: 91.1019287109375
nll mean: 89.38038635253906 	 kl mean: 1.7217345237731934 	 loss mean: 91.10211181640625
nll mean: 89.29693603515625 	 kl mean: 1.7217345237731934 	 loss mean: 91.01866912841797
nll mean: 89.36341857910156 	 kl mean: 1.7217345237731934 	 loss mean: 91.08514404296875
Approximate NLL:
tensor(126.8849, device='cuda:0')
Testing took 0:00:24.694005


In [16]:
print(wd_nlls)

[tensor(126.3514, device='cuda:0'), tensor(127.2843, device='cuda:0'), tensor(126.4913, device='cuda:0'), tensor(126.8849, device='cuda:0')]


# Free Bits

In [17]:
saved_model_files = get_model_filenames('freebits_05')
saved_model_files

['results_final/results0/freebits_05/models/sentence_vae_FreeBits_0.5_3500.pt',
 'results_final/results1/freebits_05/models/sentence_vae_FreeBits_0.5_3500.pt',
 'results_final/results2/freebits_05/models/sentence_vae_FreeBits_0.5_3500.pt',
 'results_final/results3/freebits_05/models/sentence_vae_FreeBits_0.5_3500.pt']

In [18]:
fb_nlls = get_approx_nlls(saved_model_files)

  0%|          | 0/38 [00:00<?, ?it/s]
                                      
  0%|          | 0/38 [00:00<?, ?it/s]
                                      
  0%|          | 0/38 [00:00<?, ?it/s]
                                      
  0%|          | 0/10 [00:00<?, ?it/s]
                                      .15it/s]
  0%|          | 0/38 [00:00<?, ?it/s]        
                                      .15it/s]
  0%|          | 0/38 [00:00<?, ?it/s]        
                                      .15it/s]
                                              

nll mean: 108.42697143554688 	 kl mean: 8.921194076538086 	 loss mean: 117.34815979003906
nll mean: 109.35580444335938 	 kl mean: 8.921194076538086 	 loss mean: 118.2770004272461
nll mean: 109.13043212890625 	 kl mean: 8.921194076538086 	 loss mean: 118.05162811279297
nll mean: 108.80230712890625 	 kl mean: 8.921194076538086 	 loss mean: 117.72349548339844
nll mean: 108.94507598876953 	 kl mean: 8.921194076538086 	 loss mean: 117.86627197265625


 30%|███       | 3/10 [00:00<00:00, 26.15it/s]
                                      .75it/s]
  0%|          | 0/38 [00:00<?, ?it/s]        
                                      .75it/s]
  0%|          | 0/38 [00:00<?, ?it/s]        
                                      .75it/s]
 60%|██████    | 6/10 [00:00<00:00, 25.75it/s]
                                      .45it/s]
  0%|          | 0/10 [00:00<?, ?it/s]

nll mean: 108.0947036743164 	 kl mean: 8.921194076538086 	 loss mean: 117.01589965820312
nll mean: 108.53001403808594 	 kl mean: 8.921194076538086 	 loss mean: 117.45121002197266
nll mean: 109.01799011230469 	 kl mean: 8.921194076538086 	 loss mean: 117.9391860961914
nll mean: 108.87019348144531 	 kl mean: 8.921194076538086 	 loss mean: 117.7913818359375
nll mean: 108.53302764892578 	 kl mean: 8.921194076538086 	 loss mean: 117.4542236328125



                                              
  3%|▎         | 1/38 [00:00<00:14,  2.50it/s]
                                              
 20%|██        | 2/10 [00:00<00:00, 13.66it/s]
                                              
  3%|▎         | 1/38 [00:00<00:14,  2.50it/s]

...

nll mean: 110.8900146484375 	 kl mean: 8.196907997131348 	 loss mean: 119.08692932128906
nll mean: 111.52523803710938 	 kl mean: 8.196907997131348 	 loss mean: 119.72215270996094
nll mean: 110.18514251708984 	 kl mean: 8.196907997131348 	 loss mean: 118.38204956054688
nll mean: 111.27530670166016 	 kl mean: 8.196907997131348 	 loss mean: 119.47221374511719
nll mean: 111.4046401977539 	 kl mean: 8.196907997131348 	 loss mean: 119.60154724121094



                                               
 95%|█████████▍| 36/38 [00:23<00:01,  1.34it/s][A
                                               
 90%|█████████ | 9/10 [00:00<00:00, 22.78it/s]
                                               
  0%|          | 0/10 [00:00<?, ?it/s]
                                               
 97%|█████████▋| 37/38 [00:23<00:00,  1.53it/s]
                                               
 97%|█████████▋| 37/38 [00:23<00:00,  1.53it/s]
                                               
 97%|█████████▋| 37/38 [00:23<00:00,  1.53it/s]

nll mean: 111.29986572265625 	 kl mean: 8.196907997131348 	 loss mean: 119.49678039550781
nll mean: 110.7373046875 	 kl mean: 8.196907997131348 	 loss mean: 118.9342041015625
nll mean: 110.87271118164062 	 kl mean: 8.196907997131348 	 loss mean: 119.06962585449219
nll mean: 86.32854461669922 	 kl mean: 8.369827270507812 	 loss mean: 94.6983642578125
nll mean: 85.7383041381836 	 kl mean: 8.369827270507812 	 loss mean: 94.10812377929688
nll mean: 86.28227996826172 	 kl mean: 8.369827270507812 	 loss mean: 94.6521224975586



                                               
 40%|████      | 4/10 [00:00<00:00, 31.36it/s]
                                               
 97%|█████████▋| 37/38 [00:23<00:00,  1.53it/s][A
                                               
 97%|█████████▋| 37/38 [00:23<00:00,  1.53it/s][A
                                               
 97%|█████████▋| 37/38 [00:23<00:00,  1.53it/s][A
                                               
 80%|████████  | 8/10 [00:00<00:00, 31.25it/s]
                                               
 97%|█████████▋| 37/38 [00:23<00:00,  1.53it/s][A
                                               
100%|██████████| 38/38 [00:23<00:00,  1.59it/s]

nll mean: 86.47383880615234 	 kl mean: 8.369827270507812 	 loss mean: 94.84365844726562
nll mean: 86.62020111083984 	 kl mean: 8.369827270507812 	 loss mean: 94.9900131225586
nll mean: 86.18350219726562 	 kl mean: 8.369827270507812 	 loss mean: 94.55332946777344
nll mean: 85.7130126953125 	 kl mean: 8.369827270507812 	 loss mean: 94.08284759521484
nll mean: 86.34770965576172 	 kl mean: 8.369827270507812 	 loss mean: 94.717529296875
nll mean: 85.82736206054688 	 kl mean: 8.369827270507812 	 loss mean: 94.19718933105469
nll mean: 85.65509033203125 	 kl mean: 8.369827270507812 	 loss mean: 94.02490234375
Approximate NLL:
tensor(121.1376, device='cuda:0')
Testing took 0:00:24.856624


In [19]:
print(fb_nlls)

[tensor(121.0451, device='cuda:0'), tensor(120.7842, device='cuda:0'), tensor(120.9724, device='cuda:0'), tensor(121.1376, device='cuda:0')]


# MDR

In [20]:
saved_model_files = get_model_filenames('mdr10')
saved_model_files

['results_final/results0/mdr10/models/sentence_vae_MDR_10.0_3500.pt',
 'results_final/results1/mdr10/models/sentence_vae_MDR_10.0_3500.pt',
 'results_final/results2/mdr10/models/sentence_vae_MDR_10.0_3500.pt',
 'results_final/results3/mdr10/models/sentence_vae_MDR_10.0_3500.pt']

In [21]:
mdr_nlls = get_approx_nlls(saved_model_files)

  0%|          | 0/38 [00:00<?, ?it/s]
                                      
  0%|          | 0/38 [00:00<?, ?it/s]
                                      
  0%|          | 0/38 [00:00<?, ?it/s]
                                      
  0%|          | 0/10 [00:00<?, ?it/s]
                                      .01it/s]
  0%|          | 0/38 [00:00<?, ?it/s]        
                                      .01it/s]
 30%|███       | 3/10 [00:00<00:00, 26.01it/s]

nll mean: 108.1535873413086 	 kl mean: 10.452482223510742 	 loss mean: 118.6060791015625
nll mean: 108.79361724853516 	 kl mean: 10.452482223510742 	 loss mean: 119.24609375
nll mean: 109.00895690917969 	 kl mean: 10.452482223510742 	 loss mean: 119.46144104003906
nll mean: 108.1458740234375 	 kl mean: 10.452482223510742 	 loss mean: 118.59835815429688
nll mean: 108.09036254882812 	 kl mean: 10.452482223510742 	 loss mean: 118.5428466796875


                                      
 30%|███       | 3/10 [00:00<00:00, 26.01it/s]
                                      .64it/s]
  0%|          | 0/38 [00:00<?, ?it/s]        
                                      .64it/s]
  0%|          | 0/38 [00:00<?, ?it/s]        
                                      .64it/s]
 60%|██████    | 6/10 [00:00<00:00, 25.64it/s]
                                      .36it/s]
  0%|          | 0/10 [00:00<?, ?it/s]

nll mean: 107.95036315917969 	 kl mean: 10.452482223510742 	 loss mean: 118.40284729003906
nll mean: 108.45965576171875 	 kl mean: 10.452482223510742 	 loss mean: 118.91213989257812
nll mean: 108.66999053955078 	 kl mean: 10.452482223510742 	 loss mean: 119.12247467041016
nll mean: 107.90679931640625 	 kl mean: 10.452482223510742 	 loss mean: 118.35928344726562
nll mean: 107.93218994140625 	 kl mean: 10.452482223510742 	 loss mean: 118.38468170166016



                                              
  3%|▎         | 1/38 [00:00<00:14,  2.50it/s]
                                              
 20%|██        | 2/10 [00:00<00:00, 14.03it/s]
                                              
  3%|▎         | 1/38 [00:00<00:14,  2.50it/s]

...

nll mean: 109.99429321289062 	 kl mean: 9.318066596984863 	 loss mean: 119.3123550415039
nll mean: 109.69163513183594 	 kl mean: 9.318066596984863 	 loss mean: 119.00970458984375
nll mean: 109.73252868652344 	 kl mean: 9.318066596984863 	 loss mean: 119.05059051513672
nll mean: 109.59786987304688 	 kl mean: 9.318066596984863 	 loss mean: 118.91593933105469
nll mean: 110.06913757324219 	 kl mean: 9.318066596984863 	 loss mean: 119.38720703125


                                               
 95%|█████████▍| 36/38 [00:23<00:01,  1.34it/s]
                                               [A
 60%|██████    | 6/10 [00:00<00:00, 22.44it/s]
                                               [A
 97%|█████████▋| 37/38 [00:23<00:00,  1.52it/s]
                                               
 97%|█████████▋| 37/38 [00:23<00:00,  1.52it/s]
                                               
 97%|█████████▋| 37/38 [00:23<00:00,  1.52it/s]
                                               
  0%|          | 0/10 [00:00<?, ?it/s]

nll mean: 110.22521209716797 	 kl mean: 9.318066596984863 	 loss mean: 119.54327392578125
nll mean: 110.0033950805664 	 kl mean: 9.318066596984863 	 loss mean: 119.32145690917969
nll mean: 109.956298828125 	 kl mean: 9.318066596984863 	 loss mean: 119.27436065673828
nll mean: 84.17594909667969 	 kl mean: 10.216300964355469 	 loss mean: 94.39225769042969
nll mean: 83.72074890136719 	 kl mean: 10.216300964355469 	 loss mean: 93.93704223632812
nll mean: 83.40291595458984 	 kl mean: 10.216300964355469 	 loss mean: 93.61921691894531


                                               
  0%|          | 0/10 [00:00<?, ?it/s]
                                               [A
 97%|█████████▋| 37/38 [00:23<00:00,  1.52it/s]
                                               [A
 97%|█████████▋| 37/38 [00:23<00:00,  1.52it/s]
                                               [A
 97%|█████████▋| 37/38 [00:23<00:00,  1.52it/s]
                                               [A
 40%|████      | 4/10 [00:00<00:00, 31.29it/s]
                                               [A
 97%|█████████▋| 37/38 [00:23<00:00,  1.52it/s]
                                               [A
100%|██████████| 38/38 [00:23<00:00,  1.59it/s]

nll mean: 83.48346710205078 	 kl mean: 10.216300964355469 	 loss mean: 93.69976806640625
nll mean: 83.89019775390625 	 kl mean: 10.216300964355469 	 loss mean: 94.10649871826172
nll mean: 83.97494506835938 	 kl mean: 10.216300964355469 	 loss mean: 94.19124603271484
nll mean: 83.8875961303711 	 kl mean: 10.216300964355469 	 loss mean: 94.1039047241211
nll mean: 83.99072265625 	 kl mean: 10.216300964355469 	 loss mean: 94.20702362060547
nll mean: 83.44651794433594 	 kl mean: 10.216300964355469 	 loss mean: 93.66282653808594
nll mean: 83.3628158569336 	 kl mean: 10.216300964355469 	 loss mean: 93.5791244506836
Approximate NLL:
tensor(119.5580, device='cuda:0')
Testing took 0:00:24.807150


In [22]:
print(mdr_nlls)

[tensor(119.8162, device='cuda:0'), tensor(119.8743, device='cuda:0'), tensor(120.0954, device='cuda:0'), tensor(119.5580, device='cuda:0')]


# Word dropout & Free Bits

In [23]:
saved_model_files = get_model_filenames('word_dropout_066_freebits_05')
saved_model_files

['results_final/results0/word_dropout_066_freebits_05/models/sentence_vae_FreeBits_0.5_4500.pt',
 'results_final/results1/word_dropout_066_freebits_05/models/sentence_vae_FreeBits_0.5_6000.pt',
 'results_final/results2/word_dropout_066_freebits_05/models/sentence_vae_FreeBits_0.5_6000.pt',
 'results_final/results3/word_dropout_066_freebits_05/models/sentence_vae_FreeBits_0.5_7000.pt']

In [24]:
wd_fb_nlls = get_approx_nlls(saved_model_files)

  0%|          | 0/38 [00:00<?, ?it/s]
                                      
  0%|          | 0/38 [00:00<?, ?it/s]
                                      
  0%|          | 0/38 [00:00<?, ?it/s]
                                      
  0%|          | 0/10 [00:00<?, ?it/s]
                                      .04it/s]
  0%|          | 0/38 [00:00<?, ?it/s]        
                                      .04it/s]
  0%|          | 0/38 [00:00<?, ?it/s]        
                                      .04it/s]
 30%|███       | 3/10 [00:00<00:00, 26.04it/s]

nll mean: 109.08168029785156 	 kl mean: 9.060304641723633 	 loss mean: 118.14198303222656
nll mean: 109.72286224365234 	 kl mean: 9.060304641723633 	 loss mean: 118.78317260742188
nll mean: 109.37704467773438 	 kl mean: 9.060304641723633 	 loss mean: 118.4373550415039
nll mean: 109.55122375488281 	 kl mean: 9.060304641723633 	 loss mean: 118.61151885986328
nll mean: 109.64799499511719 	 kl mean: 9.060304641723633 	 loss mean: 118.70829772949219
nll mean: 109.098876953125 	 kl mean: 9.060304641723633 	 loss mean: 118.15918731689453



                                      .73it/s]
  0%|          | 0/38 [00:00<?, ?it/s]        
                                      .73it/s]
  0%|          | 0/38 [00:00<?, ?it/s]        
                                      .73it/s]
 60%|██████    | 6/10 [00:00<00:00, 25.73it/s]
                                      .38it/s]
  3%|▎         | 1/38 [00:00<00:14,  2.50it/s]
                                              
  0%|          | 0/10 [00:00<?, ?it/s]

nll mean: 109.60954284667969 	 kl mean: 9.060304641723633 	 loss mean: 118.66984558105469
nll mean: 109.99411010742188 	 kl mean: 9.060304641723633 	 loss mean: 119.0544204711914
nll mean: 109.773681640625 	 kl mean: 9.060304641723633 	 loss mean: 118.83399200439453
nll mean: 109.61831665039062 	 kl mean: 9.060304641723633 	 loss mean: 118.67862701416016
nll mean: 127.52266693115234 	 kl mean: 8.866565704345703 	 loss mean: 136.3892364501953


                                              
  0%|          | 0/10 [00:00<?, ?it/s]
                                              
  3%|▎         | 1/38 [00:00<00:14,  2.50it/s]
                                              
 40%|████      | 4/10 [00:00<00:00, 14.26it/s]

...

nll mean: 111.37239074707031 	 kl mean: 8.624626159667969 	 loss mean: 119.99702453613281
nll mean: 112.05016326904297 	 kl mean: 8.624626159667969 	 loss mean: 120.67478942871094
nll mean: 111.67080688476562 	 kl mean: 8.624626159667969 	 loss mean: 120.29542541503906
nll mean: 111.44898986816406 	 kl mean: 8.624626159667969 	 loss mean: 120.07361602783203
nll mean: 112.20767974853516 	 kl mean: 8.624626159667969 	 loss mean: 120.83230590820312



                                               
 95%|█████████▍| 36/38 [00:23<00:01,  1.34it/s][A
                                               
 90%|█████████ | 9/10 [00:00<00:00, 22.33it/s]
                                               
  0%|          | 0/10 [00:00<?, ?it/s]
                                               
 97%|█████████▋| 37/38 [00:23<00:00,  1.51it/s]
                                               
 97%|█████████▋| 37/38 [00:23<00:00,  1.51it/s]
                                               
 97%|█████████▋| 37/38 [00:23<00:00,  1.51it/s]

nll mean: 111.65900421142578 	 kl mean: 8.624626159667969 	 loss mean: 120.28363037109375
nll mean: 112.66665649414062 	 kl mean: 8.624626159667969 	 loss mean: 121.29129028320312
nll mean: 111.58543395996094 	 kl mean: 8.624626159667969 	 loss mean: 120.21005249023438
nll mean: 84.99127960205078 	 kl mean: 9.724547386169434 	 loss mean: 94.7158203125
nll mean: 85.06403350830078 	 kl mean: 9.724547386169434 	 loss mean: 94.78857421875
nll mean: 85.0241470336914 	 kl mean: 9.724547386169434 	 loss mean: 94.74870300292969



                                               
 40%|████      | 4/10 [00:00<00:00, 31.29it/s]
                                               
 97%|█████████▋| 37/38 [00:23<00:00,  1.51it/s][A
                                               
 97%|█████████▋| 37/38 [00:23<00:00,  1.51it/s][A
                                               
 97%|█████████▋| 37/38 [00:23<00:00,  1.51it/s][A
                                               
 80%|████████  | 8/10 [00:00<00:00, 31.31it/s]
                                               
 97%|█████████▋| 37/38 [00:23<00:00,  1.51it/s][A
                                               
100%|██████████| 38/38 [00:23<00:00,  1.59it/s]

nll mean: 85.09728240966797 	 kl mean: 9.724547386169434 	 loss mean: 94.82183074951172
nll mean: 85.50777435302734 	 kl mean: 9.724547386169434 	 loss mean: 95.2323226928711
nll mean: 85.16156768798828 	 kl mean: 9.724547386169434 	 loss mean: 94.8861083984375
nll mean: 84.94012451171875 	 kl mean: 9.724547386169434 	 loss mean: 94.6646728515625
nll mean: 85.30917358398438 	 kl mean: 9.724547386169434 	 loss mean: 95.03372192382812
nll mean: 84.58796691894531 	 kl mean: 9.724547386169434 	 loss mean: 94.31251525878906
nll mean: 85.16664123535156 	 kl mean: 9.724547386169434 	 loss mean: 94.89118957519531
Approximate NLL:
tensor(121.8895, device='cuda:0')
Testing took 0:00:24.889178


In [25]:
print(wd_fb_nlls)

[tensor(121.7690, device='cuda:0'), tensor(121.3110, device='cuda:0'), tensor(121.2281, device='cuda:0'), tensor(121.8895, device='cuda:0')]


# Word dropout & MDR

In [26]:
saved_model_files = get_model_filenames('word_dropout_066_mdr_10')
saved_model_files

['results_final/results0/word_dropout_066_mdr_10/models/sentence_vae_MDR_10.0_6000.pt',
 'results_final/results1/word_dropout_066_mdr_10/models/sentence_vae_MDR_10.0_6000.pt',
 'results_final/results2/word_dropout_066_mdr_10/models/sentence_vae_MDR_10.0_6000.pt',
 'results_final/results3/word_dropout_066_mdr_10/models/sentence_vae_MDR_10.0_4500.pt']

In [27]:
wd_mdr_nlls = get_approx_nlls(saved_model_files)

  0%|          | 0/38 [00:00<?, ?it/s]
                                      
  0%|          | 0/38 [00:00<?, ?it/s]
                                      
  0%|          | 0/38 [00:00<?, ?it/s]
                                      
  0%|          | 0/10 [00:00<?, ?it/s]
                                      .94it/s]
  0%|          | 0/38 [00:00<?, ?it/s]        
                                      .94it/s]
 30%|███       | 3/10 [00:00<00:00, 25.94it/s]

nll mean: 108.67070007324219 	 kl mean: 10.019697189331055 	 loss mean: 118.69039916992188
nll mean: 108.29741668701172 	 kl mean: 10.019697189331055 	 loss mean: 118.31710815429688
nll mean: 108.79986572265625 	 kl mean: 10.019697189331055 	 loss mean: 118.81956481933594
nll mean: 108.36072540283203 	 kl mean: 10.019697189331055 	 loss mean: 118.38041687011719
nll mean: 108.55281066894531 	 kl mean: 10.019697189331055 	 loss mean: 118.572509765625


                                      
 30%|███       | 3/10 [00:00<00:00, 25.94it/s]
                                      .50it/s]
  0%|          | 0/38 [00:00<?, ?it/s]        
                                      .50it/s]
  0%|          | 0/38 [00:00<?, ?it/s]        
                                      .50it/s]
 60%|██████    | 6/10 [00:00<00:00, 25.50it/s]
                                      .28it/s]
  0%|          | 0/10 [00:00<?, ?it/s]

nll mean: 108.08068084716797 	 kl mean: 10.019697189331055 	 loss mean: 118.10037994384766
nll mean: 108.53858947753906 	 kl mean: 10.019697189331055 	 loss mean: 118.55828857421875
nll mean: 108.086181640625 	 kl mean: 10.019697189331055 	 loss mean: 118.10588073730469
nll mean: 108.47161102294922 	 kl mean: 10.019697189331055 	 loss mean: 118.4913101196289
nll mean: 108.25113677978516 	 kl mean: 10.019697189331055 	 loss mean: 118.27082824707031


                                              
  3%|▎         | 1/38 [00:00<00:14,  2.49it/s]
                                              
  0%|          | 0/10 [00:00<?, ?it/s]
                                              
 20%|██        | 2/10 [00:00<00:00, 13.56it/s]

...

nll mean: 111.13217163085938 	 kl mean: 9.077619552612305 	 loss mean: 120.20979309082031
nll mean: 111.74407958984375 	 kl mean: 9.077619552612305 	 loss mean: 120.82170104980469
nll mean: 111.40707397460938 	 kl mean: 9.077619552612305 	 loss mean: 120.48469543457031
nll mean: 111.62274932861328 	 kl mean: 9.077619552612305 	 loss mean: 120.70036315917969
nll mean: 112.04039001464844 	 kl mean: 9.077619552612305 	 loss mean: 121.11800384521484



                                               
 95%|█████████▍| 36/38 [00:23<00:01,  1.33it/s][A
                                               
 90%|█████████ | 9/10 [00:00<00:00, 22.31it/s]
                                               
 97%|█████████▋| 37/38 [00:23<00:00,  1.51it/s]
                                               
 97%|█████████▋| 37/38 [00:23<00:00,  1.51it/s]
                                               
 97%|█████████▋| 37/38 [00:23<00:00,  1.51it/s]
                                               
  0%|          | 0/10 [00:00<?, ?it/s]

nll mean: 111.23815155029297 	 kl mean: 9.077619552612305 	 loss mean: 120.3157730102539
nll mean: 111.4344711303711 	 kl mean: 9.077619552612305 	 loss mean: 120.5120849609375
nll mean: 111.26554870605469 	 kl mean: 9.077619552612305 	 loss mean: 120.3431625366211
nll mean: 85.7601318359375 	 kl mean: 10.26272964477539 	 loss mean: 96.02285766601562
nll mean: 85.16584777832031 	 kl mean: 10.26272964477539 	 loss mean: 95.42857360839844
nll mean: 84.38825988769531 	 kl mean: 10.26272964477539 	 loss mean: 94.65098571777344


                                               
  0%|          | 0/10 [00:00<?, ?it/s]
                                               [A
 97%|█████████▋| 37/38 [00:23<00:00,  1.51it/s]
                                               [A
 97%|█████████▋| 37/38 [00:23<00:00,  1.51it/s]
                                               [A
 97%|█████████▋| 37/38 [00:23<00:00,  1.51it/s]
                                               [A
 40%|████      | 4/10 [00:00<00:00, 31.02it/s]
                                               [A
 97%|█████████▋| 37/38 [00:23<00:00,  1.51it/s]
                                               [A
100%|██████████| 38/38 [00:23<00:00,  1.59it/s]

nll mean: 85.55635833740234 	 kl mean: 10.26272964477539 	 loss mean: 95.81908416748047
nll mean: 85.28278350830078 	 kl mean: 10.26272964477539 	 loss mean: 95.54551696777344
nll mean: 85.6299057006836 	 kl mean: 10.26272964477539 	 loss mean: 95.89263153076172
nll mean: 84.98606872558594 	 kl mean: 10.26272964477539 	 loss mean: 95.2488021850586
nll mean: 85.98330688476562 	 kl mean: 10.26272964477539 	 loss mean: 96.24603271484375
nll mean: 85.8359603881836 	 kl mean: 10.26272964477539 	 loss mean: 96.09869384765625
nll mean: 85.80473327636719 	 kl mean: 10.26272964477539 	 loss mean: 96.06746673583984
Approximate NLL:
tensor(121.6265, device='cuda:0')
Testing took 0:00:24.873877


In [13]:
print(vanilla_nlls)

[tensor(126.9911, device='cuda:0'), tensor(127.1313, device='cuda:0'), tensor(127.0736, device='cuda:0'), tensor(126.7324, device='cuda:0')]


In [16]:
print(wd_nlls)

[tensor(126.3514, device='cuda:0'), tensor(127.2843, device='cuda:0'), tensor(126.4913, device='cuda:0'), tensor(126.8849, device='cuda:0')]


In [19]:
print(fb_nlls)

[tensor(121.0451, device='cuda:0'), tensor(120.7842, device='cuda:0'), tensor(120.9724, device='cuda:0'), tensor(121.1376, device='cuda:0')]


In [22]:
print(mdr_nlls)

[tensor(119.8162, device='cuda:0'), tensor(119.8743, device='cuda:0'), tensor(120.0954, device='cuda:0'), tensor(119.5580, device='cuda:0')]


In [25]:
print(wd_fb_nlls)

[tensor(121.7690, device='cuda:0'), tensor(121.3110, device='cuda:0'), tensor(121.2281, device='cuda:0'), tensor(121.8895, device='cuda:0')]


In [28]:
print(wd_mdr_nlls)

[tensor(120.9221, device='cuda:0'), tensor(121.1686, device='cuda:0'), tensor(120.5179, device='cuda:0'), tensor(121.6265, device='cuda:0')]


# Results

In [36]:
all_results = {
    "Vanilla": torch.tensor(vanilla_nlls),
    "Word dropout": torch.tensor(wd_nlls),
    "Free Bits": torch.tensor(fb_nlls),
    "MDR": torch.tensor(mdr_nlls),
    "Word dropout & Free Bits": torch.tensor(wd_fb_nlls),
    "Word dropout & MDR": torch.tensor(wd_mdr_nlls),
}

In [48]:
for name, res in all_results.items():
    std, mean = torch.std_mean(res)
    print(f"{name: >30} approximated NLL: \t {mean} \t +- {std}")

                       Vanilla approximated NLL: 	 126.98207092285156 	 +- 0.1761312633752823
                  Word dropout approximated NLL: 	 126.7529296875 	 +- 0.42009851336479187
                     Free Bits approximated NLL: 	 120.98483276367188 	 +- 0.14984405040740967
                           MDR approximated NLL: 	 119.83599090576172 	 +- 0.22093771398067474
      Word dropout & Free Bits approximated NLL: 	 121.54940795898438 	 +- 0.328605979681015
            Word dropout & MDR approximated NLL: 	 121.05879211425781 	 +- 0.46391791105270386
